# 03 - Train and Deploy

In [2]:
import boto3
import sagemaker
import time
from time import strftime

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto3.client("sagemaker")
region = boto_session.region_name
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]

prefix = "sagemaker-intelimage-dvc"

print(f"account: {account}")
print(f"bucket: {bucket}")
print(f"region: {region}")
print(f"role: {role}")

account: 294495367161
bucket: sagemaker-ap-south-1-294495367161
region: ap-south-1
role: arn:aws:iam::294495367161:role/service-role/AmazonSageMaker-ExecutionRole-20221204T133204


In [3]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import TensorBoardOutputConfig

Commit Hash: `efaaa7a6279dd82e5673476bc789cfd105cb7dae`

In [4]:
dvc_repo_url = "codecommit::ap-south-1://sagemaker-imageclf"
dvc_branch = "processed-dataset"

In [5]:
tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=f"s3://{bucket}/sagemaker-intel_image-logs",
    container_local_output_path="/opt/ml/output/tensorboard",
)

In [7]:
pt_estimator = PyTorch(
    image_uri="294495367161.dkr.ecr.ap-south-1.amazonaws.com/sagemaker:latest",
    base_job_name="training-intel-dataset",
    source_dir="/root/emlov2-session-12/",
    entry_point="train.py",
    role=get_execution_role(),
    py_version="py38",
    framework_version="1.12.0",
    instance_count=1,
    # instance_type="ml.g4dn.xlarge",
    # instance_type="ml.m5.4xlarge",
    instance_type="ml.c5.4xlarge",
    tensorboard_output_config=tensorboard_output_config,
    use_spot_instances=True,
    max_wait=4200,
    max_run=4000,
    environment={
        "DVC_REPO_URL": dvc_repo_url,
        "DVC_BRANCH": dvc_branch,
        "GIT_USER": "Vivek Chaudhary",
        "GIT_EMAIL": "vivek.experiotech@gmail.com",
    },
)

In [8]:
pt_estimator.fit()

2022-12-10 11:50:47 Starting - Starting the training job...
2022-12-10 11:51:11 Starting - Preparing the instances for trainingProfilerReport-1670673046: InProgress
............
2022-12-10 11:53:11 Downloading - Downloading input data...
2022-12-10 11:53:31 Training - Downloading the training image.........
2022-12-10 11:55:11 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-10 11:55:23,443 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-10 11:55:23,444 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-10 11:55:23,446 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2022-12-10 11:55:23,456 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-12-10 11:55:23,458 sagemaker_pytorch_c